In [2]:
import torch
import numpy as np
import networkx as nx
from nilearn.connectome import ConnectivityMeasure
import matplotlib.pyplot as plt

In [19]:
path_to_file = "data/UKB1000028_ts_sc2345.txt"
window_size = 10
stride = 10
top_percent = 5

signal = np.loadtxt(path_to_file, delimiter=",").T
time_len, num_nodes = signal.shape

conn_measure = ConnectivityMeasure(kind="correlation")

window_idxs = (np.expand_dims(np.arange(window_size), 0) + np.expand_dims(np.arange(time_len, step=stride), 0).T)
signal_windows = signal[window_idxs, :]
dyn_conn = np.array([conn_measure.fit_transform([w])[0] for w in signal_windows])

In [20]:
time_steps, num_nodes, _ = dyn_conn.shape

graphs = []
edges = []

for t in range(time_steps):

    conn_matrix = dyn_conn[t]
    np.fill_diagonal(conn_matrix, 0.)

    assert (conn_matrix.min() >=  -1.) and (conn_matrix.max() <= 1.)
    
    threshold = np.percentile(conn_matrix.flatten(), 100 - top_percent)  
    conn_matrix[conn_matrix < threshold] = 0.

    # conn_matrix[conn_matrix >= 0] = 1.

    # assert (conn_matrix.min() >=  0.) and (conn_matrix.max() <= 1.)
    g = nx.from_numpy_matrix(conn_matrix)

    graphs += [g]
    edges += [np.array(list(g.edges()))]

edges = np.stack(edges)

# assert len(graphs) == time_steps
# assert len(list(graphs[0].edges())) < (num_nodes * (num_nodes - 1)) // 2
# assert len(list(graphs[0].edges())) == (conn_matrix.sum() // 2)

In [47]:
num_nodes = 3
A = np.random.rand(num_nodes, num_nodes)
A = (A + A.T) / 2
np.fill_diagonal(A, 0.)

G = nx.from_numpy_matrix(A)
edges = np.array(list(G.edges()))
print(edges.shape)

(3, 2)


In [66]:
w = np.concatenate((edges[:, 0], edges[:, 1]))
c = np.concatenate((edges[:, 1], edges[:, 0]))
print(len(w), len(c))
print(w.shape)

6 6
(6,)


In [5]:
print("-{:03}".format(3))

-003


In [9]:
np.array(list(set(np.array([1., 2.]))))

array([1., 2.])